Import needed libaries

In [1]:
from dotenv import dotenv_values
import sqlalchemy
import pandas as pd
import sql_functions as sf
import matplotlib.pyplot as plt

Define schema & engine to call df from Database

In [2]:
schema = 'capstone_wildfire'
engine = sf.get_engine()

Define the table & query 

In [3]:
sql_query = f'select * from {schema}.fires_data_v1_rdy_to_clean_rows;'

Get datatable as dataframe

In [4]:
wild_fire_df = sf.get_dataframe(sql_query)

Get the count of fires per year per state 

In [5]:
# Extract the year from the 'combined_discovery_date' column
wild_fire_df['year'] = wild_fire_df['combined_discovery_date'].dt.year

# Group the DataFrame by 'state', 'year', and count the unique 'unique_id' values
fire_counts = wild_fire_df.groupby(['state_name', 'year'])['unique_id'].nunique().reset_index()

# Rename the count column
fire_counts.rename(columns={'unique_id': 'fire_count'}, inplace=True)

# Print the result
print(fire_counts)


     state_name  year  fire_count
0       Alabama  1995        4230
1       Alabama  1996        4048
2       Alabama  1997        2591
3       Alabama  1998        3566
4       Alabama  1999        4501
...         ...   ...         ...
1055    Wyoming  2011         640
1056    Wyoming  2012        1045
1057    Wyoming  2013         581
1058    Wyoming  2014         595
1059    Wyoming  2015         632

[1060 rows x 3 columns]


Get the count of fires per year per category 

In [6]:
# Extract the year from the 'combined_discovery_date' column
wild_fire_df['year'] = wild_fire_df['combined_discovery_date'].dt.year

# Group the DataFrame by 'state', 'year', and count the unique 'unique_id' values
fire_counts_category = wild_fire_df.groupby(['cause_descr', 'year'])['unique_id'].nunique().reset_index()

# Rename the count column
fire_counts_category.rename(columns={'unique_id': 'fire_count_category'}, inplace=True)

# Print the result
print(fire_counts_category)

    cause_descr  year  fire_count_category
0         Arson  1995                12648
1         Arson  1996                10205
2         Arson  1997                10936
3         Arson  1998                14102
4         Arson  1999                19282
..          ...   ...                  ...
268   Structure  2011                  240
269   Structure  2012                  357
270   Structure  2013                  237
271   Structure  2014                  284
272   Structure  2015                  335

[273 rows x 3 columns]


Avg ha burned per category per year 

In [7]:
# Extract the year from the 'combined_discovery_date' column
wild_fire_df['year'] = wild_fire_df['combined_discovery_date'].dt.year

# Group the DataFrame by 'cause_descr' and 'year', then calculate the average 'fire_size' per group
avg_burned_ha_per_category_per_year = wild_fire_df.groupby(['cause_descr', 'year'])['fire_size'].mean().reset_index()

# Rename the 'fire_size' column to 'avg_burned_ha'
avg_burned_ha_per_category_per_year.rename(columns={'fire_size': 'avg_burned_ha'}, inplace=True)

# Print the result
print(avg_burned_ha_per_category_per_year)

    cause_descr  year  avg_burned_ha
0         Arson  1995      22.933972
1         Arson  1996      50.464074
2         Arson  1997      16.194694
3         Arson  1998      21.475740
4         Arson  1999      36.712989
..          ...   ...            ...
268   Structure  2011       4.420292
269   Structure  2012       4.300644
270   Structure  2013       3.388017
271   Structure  2014      86.505387
272   Structure  2015     214.583525

[273 rows x 3 columns]


Number of fire_class by category by Year

In [8]:
# Filter the data to only include rows where 'fire_size_class' is 'A'
filtered_data = wild_fire_df[wild_fire_df['fire_size_class'] == 'A']

# Group the filtered data by the year and calculate the sum of 'fire_size' for each year
yearly_sum = filtered_data.groupby('year')['fire_size'].sum().reset_index()

# Count the number of occurrences of 'A' in the 'fire_size_class' column for each year
yearly_A_count = wild_fire_df[wild_fire_df['fire_size_class'] == 'A'].groupby('year')['fire_size_class'].count().reset_index()

# Merge the two DataFrames on the 'year' column
result = pd.merge(yearly_sum, yearly_A_count, on='year')

# Calculate the final result by dividing the sum of 'fire_size' by the count of 'A's for each year
result['result'] = result['fire_size'] / result['fire_size_class']

# Display the result DataFrame
print(result)


    year    fire_size  fire_size_class    result
0   1995  2907.145000            23912  0.121577
1   1996  2968.250000            25119  0.118168
2   1997  2597.470000            21662  0.119909
3   1998  2836.700000            23406  0.121195
4   1999  3399.385300            27560  0.123345
5   2000  3841.495000            33060  0.116198
6   2001  3735.989000            31129  0.120016
7   2002  3282.973759            27476  0.119485
8   2003  3276.026580            28261  0.115920
9   2004  3231.868990            27354  0.118150
10  2005  3422.344000            28483  0.120154
11  2006  4126.296600            34049  0.121187
12  2007  4024.919000            33839  0.118943
13  2008  3387.711960            27848  0.121650
14  2009  3215.613280            27165  0.118373
15  2010  3413.650890            28714  0.118885
16  2011  3333.277500            28677  0.116235
17  2012  3272.380000            27346  0.119666
18  2013  3322.297010            28004  0.118637
19  2014  3202.81370

Total per category 

Combine all df into one

Try again

In [9]:
import pandas as pd
import matplotlib.pyplot as plt

# Filter the dataset for the year 2015
wild_fire_2015 = wild_fire_df[wild_fire_df['year'] == 2015]

# Filter the dataset for the previous year (2014)
wild_fire_2014 = wild_fire_df[wild_fire_df['year'] == 2014]

# Group by state and count the number of fires for 2015 and 2014
state_counts_2015 = wild_fire_2015['state_name'].value_counts().sort_values(ascending=False)
state_counts_2014 = wild_fire_2014['state_name'].value_counts().sort_values(ascending=False)

# Take the top 3 states for 2015
top_3_states_2015 = state_counts_2015.head(3)

# Find the state with the highest count in 2014
highest_count_state_2014 = state_counts_2014.index[0]

# Create a new DataFrame in the desired format for 2015
top_3_states_df = pd.DataFrame({
    'category': ['Current Year'] * 3,  # All 3 states in the same year
    'state': top_3_states_2015.index.tolist(),
    'Total Fires (Current Year)': top_3_states_2015.values.tolist()
})

# Calculate the percentage difference for each state and multiply by 100 to express as a percentage
top_3_states_df['Percentage Difference'] = [
    ((count_2015 - state_counts_2014.get(highest_count_state_2014, 0)) / state_counts_2014.get(highest_count_state_2014, 1)) * 100
    for count_2015 in top_3_states_df['Total Fires (Current Year)']
]

# Print the DataFrame
top_3_states_df


,category,state,Total Fires (Current Year),Percentage Difference
0,Current Year,Texas,8304,-2.740689
1,Current Year,California,7365,-13.738580
2,Current Year,Kansas,5887,-31.049426


In [10]:
import pandas as pd

def create_df_ct(wildfire_df, input_year):

  # Get fires for current year 
  current_year = wildfire_df[wildfire_df['combined_discovery_date'].dt.year == input_year].groupby('state_name')['unique_id'].count().sort_values(ascending=False).head(3)
  current_year = pd.DataFrame({'state_name': current_year.index, 'fire_count': current_year, 'category': 'Current Year'})

  # Get fires for previous year
  previous_year = wildfire_df[wildfire_df['combined_discovery_date'].dt.year == input_year - 1].groupby('state_name')['unique_id'].count().sort_values(ascending=False).head(3)
  previous_year = pd.DataFrame({'state_name': previous_year.index, 'fire_count': previous_year, 'category': 'Previous Year'})

 # Rest of code...

  result = pd.concat([current_year, previous_year])

  return result

In [11]:
create_df_ct(wild_fire_df, 2015)


,state_name,fire_count,category
state_name,,,
Texas,Texas,8304,Current Year
California,California,7365,Current Year
Kansas,Kansas,5887,Current Year
Texas,Texas,8538,Previous Year
California,California,6494,Previous Year
North Carolina,North Carolina,4587,Previous Year


Added the percentage difference for the previous year

In [12]:
import pandas as pd

def create_df_count(wildfire_df, input_year):

  # Get fires for current and previous year
  current_year = wildfire_df[wildfire_df['combined_discovery_date'].dt.year == input_year].groupby('state_name')['unique_id'].count().sort_values(ascending=False).head(3)
  previous_year = wildfire_df[wildfire_df['combined_discovery_date'].dt.year == input_year - 1].groupby('state_name')['unique_id'].count().sort_values(ascending=False).head(3)

  current_year = pd.DataFrame({'fire_count': current_year, 'category': 'Current Year'})
  previous_year = pd.DataFrame({'fire_count': previous_year, 'category': 'Previous Year'})

  # Calculate percentage difference
  for i in range(3):
    current_top = current_year.iloc[i]['fire_count']
    prev_top = previous_year.iloc[i]['fire_count']
    perc_diff = (current_top - prev_top) / prev_top * 100
    current_year.at[current_year.index[i], 'perc_diff'] = perc_diff

  # Set index name to state_name
  current_year.index.name = 'state_name'
  previous_year.index.name = 'state_name'

  # Rest of code...

  result = pd.concat([current_year, previous_year])

  return result

In [24]:
create_df_count(wild_fire_df, 2007)

,fire_count,category,perc_diff
state_name,,,
California,10151,Current Year,-32.421277
Georgia,8717,Current Year,-2.330532
North Carolina,7284,Current Year,-11.933261
Texas,15021,Previous Year,NaN
Georgia,8925,Previous Year,NaN
California,8271,Previous Year,NaN


Calculating the burned Acres

In [14]:
import pandas as pd 

def create_df_acres(wildfire_df, input_year):

  # Get burned acres for current and previous year
  current_year = wildfire_df[wildfire_df['combined_discovery_date'].dt.year == input_year].groupby('state_name')['fire_size'].sum().sort_values(ascending=False).head(3)

  previous_year = wildfire_df[wildfire_df['combined_discovery_date'].dt.year == input_year - 1].groupby('state_name')['fire_size'].sum().sort_values(ascending=False).head(3)
  
  current_year = pd.DataFrame({'burned_acres': current_year, 'category': 'Current Year'})  
  previous_year = pd.DataFrame({'burned_acres': previous_year, 'category': 'Previous Year'})

  # Calculate percentage difference
  for i in range(3):
    current_top = current_year.iloc[i]['burned_acres']
    prev_top = previous_year.iloc[i]['burned_acres']
    perc_diff = (current_top - prev_top) / prev_top * 100
    current_year.at[current_year.index[i], 'perc_diff'] = perc_diff  

  # Set index name to state_name
  current_year.index.name = 'state_name'
  previous_year.index.name = 'state_name'

  current_year['burned_acres'] = current_year['burned_acres'].apply(lambda x: '{:,.0f}'.format(x))
  previous_year['burned_acres'] = previous_year['burned_acres'].apply(lambda x: '{:,.0f}'.format(x))

  #keep coding

  result = pd.concat([current_year, previous_year])

  return result

In [23]:
create_df_acres(wild_fire_df, 2010)

,burned_acres,category,perc_diff
state_name,,,
Alaska,"1,137,533",Current Year,-61.434262
Idaho,"643,868",Current Year,15.432316
Texas,"238,202",Current Year,-47.688150
Alaska,"2,949,594",Previous Year,NaN
Texas,"557,788",Previous Year,NaN
California,"455,350",Previous Year,NaN


Create the df for burn_time reason's 

In [16]:
import pandas as pd

def create_df_avg_burning_time(wildfire_df, input_year):

  # Get average burning time per unique ID for current and previous year
  current_year = wildfire_df[wildfire_df['combined_discovery_date'].dt.year == input_year].groupby('state_name').agg({'burning_time': 'sum', 'unique_id': 'count'})
  current_year['avg_burning_time'] = current_year['burning_time'] / current_year['unique_id']
  current_year = current_year['avg_burning_time'].sort_values(ascending=False).head(3)

  previous_year = wildfire_df[wildfire_df['combined_discovery_date'].dt.year == input_year - 1].groupby('state_name').agg({'burning_time': 'sum', 'unique_id': 'count'}) 
  previous_year['avg_burning_time'] = previous_year['burning_time'] / previous_year['unique_id']
  previous_year = previous_year['avg_burning_time'].sort_values(ascending=False).head(3)

  current_year = pd.DataFrame({'avg_burning_time': current_year, 'category': 'Current Year'})
  previous_year = pd.DataFrame({'avg_burning_time': previous_year, 'category': 'Previous Year'})

  # Calculate percentage difference
  
  for i in range(3):
    current_top = current_year.iloc[i]['avg_burning_time']
    prev_top = previous_year.iloc[i]['avg_burning_time']
    perc_diff = (current_top - prev_top) / prev_top * 100
    current_year.at[current_year.index[i], 'perc_diff'] = perc_diff

  # Set index name to state_name
  current_year.index.name = 'state_name' 
  previous_year.index.name = 'state_name'

  current_year['avg_burning_time'] = current_year['avg_burning_time'].apply(lambda x: '{:,.2f}'.format(x))
  previous_year['avg_burning_time'] = previous_year['avg_burning_time'].apply(lambda x: '{:,.2f}'.format(x))

  result = pd.concat([current_year, previous_year])

  return result

In [17]:
create_df_avg_burning_time(wild_fire_df, 2015)

,avg_burning_time,category,perc_diff
state_name,,,
Alaska,23.80,Current Year,-42.923370
Washington,9.79,Current Year,25.977013
Hawaii,6.75,Current Year,34.562126
New York,41.71,Previous Year,NaN
Washington,7.77,Previous Year,NaN
New Jersey,5.02,Previous Year,NaN


In [28]:
import pandas as pd

def create_df_avg_burning_time(wildfire_df, input_year):

  # Current year
  current_year = wildfire_df[wildfire_df['combined_discovery_date'].dt.year == input_year].groupby('state_name').agg({'burning_time': 'sum', 'unique_id': 'count'})
  current_year['avg_burning_time'] = current_year['burning_time'] / current_year['unique_id']
  current_year = current_year['avg_burning_time'].sort_values(ascending=False).head(3)

  # Previous year 
  previous_year = wildfire_df[wildfire_df['combined_discovery_date'].dt.year == input_year - 1].groupby('state_name').agg({'burning_time': 'sum', 'unique_id': 'count'})
  previous_year['avg_burning_time'] = previous_year['burning_time'] / previous_year['unique_id']
  previous_year = previous_year['avg_burning_time'].sort_values(ascending=False).head(3)

  # Past 5 years
  start_year = input_year - 5
  end_year = input_year
  past_5_years = wildfire_df[(wildfire_df['combined_discovery_date'].dt.year >= start_year) &
                            (wildfire_df['combined_discovery_date'].dt.year <= end_year)].groupby('state_name').agg({'burning_time': 'sum', 'unique_id': 'count'})
  past_5_years['avg_burning_time'] = past_5_years['burning_time'] / past_5_years['unique_id']
  past_5_years = past_5_years['avg_burning_time'].sort_values(ascending=False).head(3)

  # Past 10 years
  start_year = input_year - 10
  end_year = input_year
  past_10_years = wildfire_df[(wildfire_df['combined_discovery_date'].dt.year >= start_year) &
                             (wildfire_df['combined_discovery_date'].dt.year <= end_year)].groupby('state_name').agg({'burning_time': 'sum', 'unique_id': 'count'})
  past_10_years['avg_burning_time'] = past_10_years['burning_time'] / past_10_years['unique_id']
  past_10_years = past_10_years['avg_burning_time'].sort_values(ascending=False).head(3)

  
  # Build dataframes
  current_year_df = pd.DataFrame({'avg_burning_time': current_year, 'category': 'Current Year'})
  previous_year_df = pd.DataFrame({'avg_burning_time': previous_year, 'category': 'Previous Year'})
  past_5_years_df = pd.DataFrame({'avg_burning_time': past_5_years, 'category': 'Past 5 Years'})
  past_10_years_df = pd.DataFrame({'avg_burning_time': past_10_years, 'category': 'Past 10 Years'})

  result = pd.concat([current_year_df, previous_year_df, past_5_years_df, past_10_years_df])

  return result

In [30]:
create_df_avg_burning_time(wild_fire_df,2005)

,avg_burning_time,category
state_name,,
Alaska,12.437803,Current Year
Idaho,7.856643,Current Year
Wyoming,2.707692,Current Year
Alaska,11.873199,Previous Year
Idaho,2.102812,Previous Year
New Mexico,1.078926,Previous Year
Alaska,7.635441,Past 5 Years
Idaho,4.308827,Past 5 Years
Wyoming,2.106772,Past 5 Years


In [58]:
import pandas as pd
import numpy as np

def create_trend_df(wildfire_df, input_year):

  # Convert date column to datetime 
  wildfire_df['combined_discovery_date'] = pd.to_datetime(wildfire_df['combined_discovery_date'])

  # Get total fires per state per year
  yearly_counts = wildfire_df.groupby(['state_name', wildfire_df['combined_discovery_date'].dt.year])['unique_id'].count().reset_index(name='unique_id')

  # Get top 10 states by average fires per year
  state_avg = yearly_counts.groupby('state_name')['unique_id'].mean().sort_values(ascending=False).head(10)
  
  # Get top state names
  top_states = state_avg.index

  # Create output DataFrame
  output = pd.DataFrame({
    'category': ['20 Year AVG']*len(top_states),
    'state': top_states,
    'total_avg': state_avg.values
  })

  return output

In [59]:
create_trend_df(wild_fire_df,2015)

,category,state,total_avg
0,20 Year AVG,California,7703.809524
1,20 Year AVG,Georgia,6986.333333
2,20 Year AVG,Texas,6594.619048
3,20 Year AVG,North Carolina,4525.428571
4,20 Year AVG,New York,3812.666667
5,20 Year AVG,Florida,3712.380952
6,20 Year AVG,Mississippi,3208.619048
7,20 Year AVG,South Carolina,3159.190476
8,20 Year AVG,Alabama,2994.571429
9,20 Year AVG,Arizona,2806.142857


In [79]:
import pandas as pd
def create_trend_df_avgburn(wildfire_df, input_year):

  # Calculate 20 year averages 
  yearly_data = wildfire_df.groupby(['state_name', wildfire_df['combined_discovery_date'].dt.year]).agg({'burning_time':'sum', 'unique_id':'count'}).reset_index()

  yearly_data['avg_burning_time'] = yearly_data['burning_time'] / yearly_data['unique_id']

  state_avg = yearly_data.groupby('state_name')['avg_burning_time'].mean().sort_values(ascending=False).head(10)

  # Output as numbers not lists
  output = pd.DataFrame({
    'category': ['20 Year AVG']*len(state_avg),
    'state': state_avg.index,
    'total_avg': state_avg.values
  })

  return output

In [80]:
create_trend_df_avgburn(wild_fire_df,2015)

,category,state,total_avg
0,20 Year AVG,Alaska,6.760629
1,20 Year AVG,Idaho,3.993359
2,20 Year AVG,Washington,3.074816
3,20 Year AVG,Hawaii,2.666519
4,20 Year AVG,New York,2.053536
5,20 Year AVG,Wyoming,1.849515
6,20 Year AVG,Montana,1.591331
7,20 Year AVG,New Mexico,1.266968
8,20 Year AVG,Oregon,1.235381
9,20 Year AVG,Utah,1.214334


In [77]:
import pandas as pd

def create_trend_df_avgsize(wildfire_df):

  # Calculate averages only for fire_size
  yearly_data = wildfire_df.groupby(['state_name', wildfire_df['combined_discovery_date'].dt.year]).agg({'fire_size':'sum', 'unique_id':'count'}).reset_index()

  yearly_data['avg_fire_size'] = yearly_data['fire_size'] / yearly_data['unique_id']

  state_avg_size = yearly_data.groupby('state_name')['avg_fire_size'].mean().sort_values(ascending=False).head(10)

  # Construct output DataFrame
  output = pd.DataFrame({
    'category': ['20 Year AVG Size']*len(state_avg_size), 
    'state': state_avg_size.index,
    'total_avg': state_avg_size.values
  })

  return output

In [78]:
create_trend_df_avgsize(wild_fire_df)

,category,state,total_avg
0,20 Year AVG Size,Alaska,2460.209963
1,20 Year AVG Size,Kansas,606.498086
2,20 Year AVG Size,Nevada,477.392100
3,20 Year AVG Size,Idaho,399.423410
4,20 Year AVG Size,Nebraska,277.029829
5,20 Year AVG Size,Hawaii,233.709699
6,20 Year AVG Size,New Mexico,168.187850
7,20 Year AVG Size,Oregon,161.329295
8,20 Year AVG Size,Wyoming,160.937520
9,20 Year AVG Size,Washington,145.016463
